In [53]:
import math
import pygame
import tkinter as tk
from tkinter import *
import random
import matplotlib.pyplot as plt
import timeit

In [54]:
# Bodies are considered to be circular and 2-dimensional
# They are placed in relation to an origin (0, 0) on a 2d coordinate grid

class Body:
    def __init__(self, mass, radius, x_coordinate, y_coordinate, name, is_immobile, x_velocity=0, y_velocity=0):
        # Standard physics symbols will be used, e.g. the lowercase letter m for mass
        # Standard physics units will also be used, e.g. the unit kg for mass
        self.m = mass
        self.r = radius
        self.name = name
        self.is_immobile = is_immobile
        self.vx, self.vy = x_velocity, y_velocity
        self.v = math.sqrt(self.vx**2+self.vy**2)
        self.ax, self.ay = 0, 0
        self.a = math.sqrt(self.ax**2+self.ay**2)
        self.x, self.y = x_coordinate, y_coordinate
        self.prev_x, self.prev_y = self.x, self.y
        
        
    # The body is moved along its velocity vectors for t seconds.
    # If it is considered immobile its velocity and acceleration vectors are set to zero.
    def move(self, t):
        self.prev_x, self.prev_y = self.x, self.y
        if self.is_immobile:
            self.vx, self.vy, self.v, self.a, self.ax, self.ay = 0, 0, 0, 0, 0, 0
        self.x += self.vx*t
        self.y += self.vy*t
        


In [55]:
class BarnesHutNode:
    def __init__(self, contained_bodies, center_x_coordinate, center_y_coordinate, width):
        self.contained_bodies = contained_bodies
        self.num_contained_bodies = len(contained_bodies)
        self.cx, self.cy = center_x_coordinate, center_y_coordinate
        self.width = width
        
        self.m, self.cmx, self.cmy = None, None, None
        self.max_radius = None
        self.tl, self.tr, self.bl, self.br = None, None, None, None
    
    
class BarnesHutTree:
    def __init__(self):
        self.contained_bodies = []
        self.root = None
            
        
    def set_root(self):
        # Find the smallest square that contains all the bodies. This will be the root node of the tree.
        min_x, max_x, min_y, max_y = math.inf, -math.inf, math.inf, -math.inf
        for b in self.contained_bodies:
            if b.x < min_x:
                min_x = b.x
            if b.x > max_x:
                max_x = b.x
            if b.y < min_y:
                min_y = b.y
            if b.y > max_y:
                max_y = b.y
                
        center_x_coordinate = (min_x+max_x)/2
        center_y_coordinate = (min_y+max_y)/2
        width = max(max_x-min_x, max_y-min_y)
        self.root = BarnesHutNode(self.contained_bodies, center_x_coordinate, center_y_coordinate, width)
        
        
    def find_root_box(self, contained_bodies):
        # Find the smallest square that contains all the bodies. This will be the root node of the tree.
        self.contained_bodies = contained_bodies
        min_x, max_x, min_y, max_y = math.inf, -math.inf, math.inf, -math.inf
        for b in self.contained_bodies:
            if b.x < min_x:
                min_x = b.x
            if b.x > max_x:
                max_x = b.x
            if b.y < min_y:
                min_y = b.y
            if b.y > max_y:
                max_y = b.y
                
        center_x_coordinate = (min_x+max_x)/2
        center_y_coordinate = (min_y+max_y)/2
        width = math.ceil(max(max_x-min_x, max_y-min_y)+1)
        return center_x_coordinate, center_y_coordinate, width
            

    def build_node(self, contained_bodies, center_x, center_y, width):
        node = BarnesHutNode(contained_bodies, center_x, center_y, width)
        
        if node.num_contained_bodies > 1:
            top_left_bodies, top_right_bodies, bottom_left_bodies, bottom_right_bodies = [], [], [], []
            max_radius = 0
            for b in node.contained_bodies:
                if b.r > max_radius:
                    max_radius = b.r
                    
                if b.x <= node.cx and b.x >= node.cx-node.width/2 and b.y >= node.cy and b.y <= node.cy+node.width/2:
                    top_left_bodies.append(b)
                elif b.x >= node.cx and b.x <= node.cx+node.width/2 and b.y >= node.cy and b.y <= node.cy+node.width/2:
                    top_right_bodies.append(b)
                elif b.x <= node.cx and b.x >= node.cx-node.width/2 and b.y <= node.cy and b.y >= node.cy-node.width/2:
                    bottom_left_bodies.append(b)
                elif b.x >= node.cx and b.x <= node.cx+node.width/2 and b.y <= node.cy and b.y >= node.cy-node.width/2:
                    bottom_right_bodies.append(b)
            
            node.tl = self.build_node(top_left_bodies, node.cx-node.width/4, node.cy+node.width/4, width/2)
            node.tr = self.build_node(top_right_bodies, node.cx+node.width/4, node.cy+node.width/4, width/2)
            node.bl = self.build_node(bottom_left_bodies, node.cx-node.width/4, node.cy-node.width/4, width/2)
            node.br = self.build_node(bottom_right_bodies, node.cx+node.width/4, node.cy-node.width/4, width/2)
        
            node.m = node.tl.m+node.tr.m+node.bl.m+node.br.m
            node.cmx = (node.tl.cmx*node.tl.m+node.tr.cmx*node.tr.m+node.bl.cmx*node.bl.m+node.br.cmx*node.br.m)/node.m
            node.cmy = (node.tl.cmy*node.tl.m+node.tr.cmy*node.tr.m+node.bl.cmy*node.bl.m+node.br.cmy*node.br.m)/node.m
            node.max_radius = max_radius
        elif node.num_contained_bodies == 1:
            body = contained_bodies[-1]
            node.m, node.cmx, node.cmy = body.m, body.x, body.y
            node.max_radius = body.r
        else:
            node.m, node.cmx, node.cmy = 0, node.cx, node.cy
            node_max_radius = 0
    
        return node        
    
    
    def build(self, contained_bodies):
        root_box_center_x, root_box_center_y, root_box_width = self.find_root_box(contained_bodies)
        self.root = self.build_node(contained_bodies, root_box_center_x, root_box_center_y, root_box_width)

In [56]:
class GravitySimulation:
    # Time step is how many seconds each body is moved along its velocity vector each simulation step
    def __init__(self, time_step):
        self.bodies = [] # A simple list to keep track of the bodies objects in the simulation
        self.num_bodies = 0 # A helper variable to avoid constant len() calls
        self.t = time_step # How long to move bodies along their velocity vectors each time step
        self.G = 6.6743e-11 # The gravitational constant, used to calculate gravitational force between bodies
        self.θ = 1 # Used to determine which bodies are "long distance" in the Barnes-Hut algorithm. Smaller values means more accuracy.
        self.num_merged = 0 # Keeps track of the total number of merged body across all time, used to give merged bodies unique names.
        self.tree = None # Stores the Barnes-Hut tree
        self.epsilon = 1 # Used for the smoothening technique
                
        
    def add_body(self, mass, radius, x_coordinate, y_coordinate, name, is_immobile, x_velocity=0, y_velocity=0):
        new_body = Body(mass, radius, x_coordinate, y_coordinate, name, is_immobile, x_velocity, y_velocity)
        self.num_bodies += 1
        self.bodies.append(new_body)
        
        
    # Calculates the sum of all gravitational forces between all pairs of objects and adjusts their acceleration and velocity vectors accordingly
    # Then the objects are moved along their velocity vectors for t time.
    def direct_sum_step(self):
        # Gravity is calculated between pairs of bodies, so all pairs of bodies must be found.
        pairs = []
        for i in range(self.num_bodies):
            for j in range(i+1, self.num_bodies):
                pairs.append((self.bodies[i], self.bodies[j]))
                self.bodies[i].ax, self.bodies[i].ay, self.bodies[i].a = 0, 0, 0
                self.bodies[j].ax, self.bodies[j].ay, self.bodies[j].a = 0, 0, 0

        for pair in pairs:
            b1, b2 = pair[0], pair[1] # Stands for body 1 and body 2. Short variable names are used for shorter code.
            d = math.sqrt((b1.x-b2.x)**2+(b1.y-b2.y)**2+self.epsilon**2) # d is the softened distance between the centers of the two bodies. 
            # Update the acceleration vectors of each body due to gravity
            # The magnitude of the acceleration due to gravity on each body
            a1 = (self.G*b2.m)/d**2
            a2 = (self.G*b1.m)/d**2
            # The angle in radians of the acceleration vector with respect to the x axis for each body (e.g. 90 degrees is North)
            # math.atan2 is used instead of math.atan as it considers the sign of the b2.y-b1.y and b2.x-b1.x terms
            θ1 = math.atan2(b2.y-b1.y, b2.x-b1.x)
            θ2 = math.atan2(b1.y-b2.y, b1.x-b2.x)
            # The acceleration vectors due to gravity on each body in the x and y axes
            a1x, a1y = a1*math.cos(θ1), a1*math.sin(θ1)
            a2x, a2y = a2*math.cos(θ2), a2*math.sin(θ2)
    
            # Add these acceleration vectors to each body in their respective direction
            b1.ax += a1x
            b1.ay += a1y
            b2.ax += a2x
            b2.ay += a2y
 
        # Finally, update each body's velocity vectors and move each body along its velocity vectors for t seconds
        for b in self.bodies:
            b.vx += b.ax*self.t
            b.vy += b.ay*self.t
            b.v = math.sqrt(b.vx**2+b.vy**2)
            b.a = math.sqrt(b.ax**2+b.ay**2)
            b.move(self.t)
                        
    
    # Merge any group of bodies that collide together into one body. Done by checking pairs of bodies in O(n²)
    def direct_sum_collision_merge(self):
        queue = self.bodies.copy()
        new_bodies_list = []
        
        while queue:
            b1 = queue[0]
            i = 1
            merged = False
            while i < len(queue):
                b2 = queue[i]
                d = math.sqrt((b1.prev_x-b2.prev_x)**2+(b1.prev_y-b2.prev_y)**2)
                a = (b1.vx-b2.vx)**2+(b1.vy-b2.vy)**2
                b = 2*((b1.prev_x-b2.prev_x)*(b1.vx-b2.vx)+(b1.prev_y-b2.prev_y)*(b1.vy-b2.vy))
                c = (b1.prev_x-b2.prev_x)**2+(b1.prev_y-b2.prev_y)**2-(b1.r+b2.r)**2
                determinant = b**2-4*a*c
                if determinant >= 0 and a != 0:
                    root_1 = (-b+math.sqrt(determinant))/(2*a)
                    root_2 = (-b-math.sqrt(determinant))/(2*a)
                    lesser_root = min(root_1, root_2)
                    larger_root = max(root_1, root_2)
                    
                if d < b1.r+b2.r or (determinant >= 0 and a != 0 and self.t >= lesser_root and larger_root > 0):
                    merged = True
                    self.num_merged += 1
                    b1 = Body(b1.m+b2.m, math.sqrt(b1.r**2+b2.r**2), (b1.x*b1.m+b2.x*b2.m)/(b1.m+b2.m), (b1.y*b1.m+b2.y*b2.m)/(b1.m+b2.m), 
                            f"Merged body {self.num_merged}", False, (b1.vx*b1.m+b2.vx*b2.m)/(b1.m+b2.m), (b1.vy*b1.m+b2.vy*b2.m)/(b1.m+b2.m))
                    del queue[i]
                    i -= 1
                i += 1
                
            if not merged:
                new_bodies_list.append(b1)
            else:
                queue.append(b1)

            del queue[0]
        
        self.bodies = new_bodies_list
        self.num_bodies = len(self.bodies)
            
    
    def merge_bodies_on_top_of_each_other(self):
        v = {}
        for b1 in self.bodies:
            if (b1.x, b1.y) in v: 
                b2 = v[(b1.x, b1.y)]
                v[(b1.x, b1.y)] = Body(b1.m+b2.m, math.sqrt(b1.r**2+b2.r**2), (b1.x*b1.m+b2.x*b2.m)/(b1.m+b2.m), (b1.y*b1.m+b2.y*b2.m)/(b1.m+b2.m), 
                                f"Merged body {self.num_merged}", False, (b1.vx*b1.m+b2.vx*b2.m)/(b1.m+b2.m), (b1.vy*b1.m+b2.vy*b2.m)/(b1.m+b2.m))
            else: v[(b1.x, b1.y)] = b1
            
        self.bodies = list(v.values())
        self.num_bodies = len(self.bodies)
        
            
    # For this to be guaranteed to work:
        # The body to orbit around must be immobile
        # There are no other bodies in the simulation
    # Changing the semi-major axis length can form elliptical orbits.
    def generate_stable_orbit(self, orbiting_body, body_to_orbit_around, semimajor_axis_length):
        b1, b2 = orbiting_body, body_to_orbit_around # Assign shorter variable names for shorter codes
        l = semimajor_axis_length # l stands for 'length', here the length of the semi-major axis
        d = math.dist((b1.x, b1.y), (b2.x, b2.y)) # d is the distance between the centers of the two bodies.
        if l < d/2: # The length of the semi-major axis is impossibly short
            raise ValueError("The Semimajor axis length cannot be less than half the distance between the bodies")
        vo = math.sqrt(self.G*b2.m*(2/d-1/l)) # The magnitude of the orbital velocity vector for the body that is orbiting
        θ = math.atan2(b2.y-b1.y, b2.x-b1.x) # The angle from the orbiting body to the body it is orbiting aroudn
        vox, voy = vo*math.cos(-math.pi/2-θ), -vo*math.sin(-math.pi/2-θ) # Extract the velocities in the x and y directions
        b1.vx, b1.vy = vox, voy # Set the orbiting body's velocities to the orbit velocities in their respective direction
        b1.v = math.sqrt(b1.vx**2+b1.vy**2)
        
        
    def build_barnes_hut_tree(self):
        self.tree = BarnesHutTree()
        self.tree.build(self.bodies)
        
        
    # Step using the Barnes-Hut algorithm
    # This has an O(n log n) complexity compared to the O(n²) complexity of the direct_sum_step() function
    # Note however that it is less accurate and slower for small numbers of bodies, since it has a significant overhead.
    def barnes_hut_step(self):        
        for b in self.bodies:
            b.a, b.ax, b.ay = 0, 0, 0
            stack = [self.tree.root]
            while stack:
                front_node = stack.pop()
                # Checks that the front node has a non-zero number of bodies
                if front_node.num_contained_bodies != 0 and not (front_node.num_contained_bodies == 1 and front_node.contained_bodies[0] == b):
                    # Softened distance between the body and the node's center of mass
                    d = math.sqrt((b.x-front_node.cmx)**2+(b.y-front_node.cmy)**2+self.epsilon**2)
                    # If this statement is true, this node is considered far away so its center of mass will be used.
                    if d != 0 and front_node.width/d < self.θ or (front_node.num_contained_bodies == 1 and front_node.contained_bodies[0] != b):
                        # The magnitude of the acceleration due to gravity
                        a = (self.G*front_node.m)/d**2
                        # The angle in radians of the acceleration vector with respect to the x axis (e.g. 90 degrees is North)
                        # math.atan2 is used instead of math.atan as it considers the sign of the delta y and delta x terms
                        θ = math.atan2(front_node.cmy-b.y, front_node.cmx-b.x)
                        # The acceleration vectors due to gravity on each body in the x and y axes
                        ax, ay = a*math.cos(θ), a*math.sin(θ)
                        # Add these acceleration vectors to the body in their respective direction
                        b.ax += ax
                        b.ay += ay
                    else: # This node is not considered far away. Its children will be enqueued.
                        for i in [front_node.tl, front_node.tr, front_node.bl, front_node.br]:
                            if i is not None: stack.append(i)
            
        for b in self.bodies:
            b.vx += b.ax*self.t
            b.vy += b.ay*self.t
            b.v = math.sqrt(b.vx**2+b.vy**2)
            b.a = math.sqrt(b.ax**2+b.ay**2)
            b.move(self.t)


    def barnes_hut_collision_merge(self):
        merged = True
        
        while merged:
            merged = False
            removed = set()
            new_bodies = []
            
            for b1 in self.bodies:
                if b1 in removed: 
                    continue
                
                stack = [self.tree.root]
                while stack:
                    front_node = stack.pop()
                    if front_node.num_contained_bodies == 1 and front_node.contained_bodies[0] != b1 and front_node.contained_bodies[0] not in removed:
                        b2 = front_node.contained_bodies[0]
                        d = math.sqrt((b1.prev_x-b2.prev_x)**2+(b1.prev_y-b2.prev_y)**2)
                        a = (b1.vx-b2.vx)**2+(b1.vy-b2.vy)**2
                        b = 2*((b1.prev_x-b2.prev_x)*(b1.vx-b2.vx)+(b1.prev_y-b2.prev_y)*(b1.vy-b2.vy))
                        c = (b1.prev_x-b2.prev_x)**2+(b1.prev_y-b2.prev_y)**2-(b1.r+b2.r)**2
                        determinant = b**2-4*a*c
                        if determinant >= 0 and a != 0:
                            root_1 = (-b+math.sqrt(determinant))/(2*a)
                            root_2 = (-b-math.sqrt(determinant))/(2*a)
                            lesser_root = min(root_1, root_2)
                            larger_root = max(root_1, root_2)
                            
                        if d < b1.r+b2.r or (determinant >= 0 and a != 0 and self.t >= lesser_root and larger_root > 0):
                            self.num_merged += 1
                            merged = True
                            removed.add(b1)
                            removed.add(b2)
                            b1 = Body(b1.m+b2.m, math.sqrt(b1.r**2+b2.r**2), (b1.x*b1.m+b2.x*b2.m)/(b1.m+b2.m), (b1.y*b1.m+b2.y*b2.m)/(b1.m+b2.m), 
                                    f"Merged body {self.num_merged}", False, (b1.vx*b1.m+b2.vx*b2.m)/(b1.m+b2.m), (b1.vy*b1.m+b2.vy*b2.m)/(b1.m+b2.m))
                        
                    elif front_node.num_contained_bodies > 1:
                        if front_node.cx+front_node.width/2 < b1.x:
                            min_x_dist_to_body = front_node.cx+front_node.width/2-b1.x
                        elif front_node.cx-front_node.width/2 > b1.x:
                            min_x_dist_to_body = front_node.cx-front_node.width/2-b1.x
                        else: 
                            min_x_dist_to_body = 0
                            
                        if front_node.cy+front_node.width/2 < b1.y:
                            min_y_dist_to_body = front_node.cy+front_node.width/2-b1.y
                        elif front_node.cy-front_node.width/2 > b1.y:
                            min_y_dist_to_body = front_node.cy-front_node.width/2-b1.y
                        else: 
                            min_y_dist_to_body = 0
                            
                        min_dist_to_body = math.sqrt(min_x_dist_to_body**2+min_y_dist_to_body**2)
                        
                        if min_dist_to_body <= b1.r+front_node.max_radius: # It's possible for the body to collide with this node.
                            stack.append(front_node.tl)
                            stack.append(front_node.tr)
                            stack.append(front_node.bl)
                            stack.append(front_node.br)
                            
                new_bodies.append(b1)
                
            self.bodies = new_bodies
            self.build_barnes_hut_tree()
            
        

In [57]:
class GUI:
    def __init__(self):
        pygame.init()
        self.fps = 60 # Stands for frames per second, equivalent to simulation steps per second
        self.sim = GravitySimulation(1) # Default time step of 1 second, changeable by the user
        self.window = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
        self.root = tk.Tk() # The backend tkinter.
        self.width, self.height = self.window.get_size() # Width and height of the window in pixels
        self.body_scaling = 1 # A simple scale factor for the size of the bodies, changeable by zooming in or out
        self.square_length = 30 # The length of the small grid squares on the screen
        self.x_offset, self.y_offset = 0, 0 # Offset of grid and bodies to allow for screen dragging
        self.arrow_scale = 1
        self.arrow_scale = 1
        self.clock = pygame.Clock()
        self.mouse_held = False
        self.paused = True
        self.adding_body = False
        self.selected_body = None
        self.centered_body = None
        self.stepping_mode = StringVar(self.root, "Stepping method: Direct Sum")
        self.collision_detection = StringVar(self.root, "Collision detection: Direct Sum")
        
        self.selected_body_info = {"name": StringVar(self.root, "-"), "m": StringVar(self.root, "-"), "r": StringVar(self.root, "-"), 
                                   "x": StringVar(self.root, "-"), "y": StringVar(self.root, "-"), "v": StringVar(self.root, "-"), 
                                   "vx": StringVar(self.root, "-"), "vy": StringVar(self.root, "-"), 
                                   "is_immobile": StringVar(self.root, "-"), "a": StringVar(self.root, "-")}
        self.body_info_entries = {}
        self.orbit_generator_inputs = {"orbiting_body": StringVar(self.root, "-"), 
                                       "body_to_orbit_around": StringVar(self.root, "-"), 
                                       "semimajor_axis_length": StringVar(self.root, "-")}
        self.orbit_generator_optionmenus = {}
        self.init_tkinter()
                    
        # ----------------------------------------
        # Add initial bodies
              
        # Earth and moon to scale  
        # self.sim.add_body(5.97219*10**24, 6378.1*10**3, 0, 0, "Earth", True)
        # self.sim.add_body(7.35*10**22, 1737.4*10**3, 384400*10**3, 0, "Moon", False)
        # self.sim.generate_stable_orbit(self.sim.bodies[1], self.sim.bodies[0], 384400*10**3)
        
        # Binary system
        self.sim.add_body(10**14, 10, -175, 0, "Body 1", False, y_velocity=2)
        self.sim.add_body(10**14, 10, 175, 0, "Body 2", False, y_velocity=-2)
        
        # self.sim.add_body(10**14, 10, -5, 0, "Body 1", False, y_velocity=0)
        # self.sim.add_body(10**14, 10, 5, 0, "Body 2", False, y_velocity=0)
        
        # Lots of bodies!
        # import random
        # random.seed(5)
        # for i in range(1000):
        #     self.sim.add_body(10**11.5, 10, random.randint(1, 1000), random.randint(1, 1000), f"Body {i}", False, x_velocity=10*(random.random()-0.5), y_velocity=10*(random.random()-0.5))
        
        self.gui_mainloop()
        
        
    def step(self):
        self.sim.merge_bodies_on_top_of_each_other()
        
        if self.stepping_mode.get() == "Stepping method: Barnes-Hut" or self.collision_detection.get() == "Collision detection: Barnes-Hut":
            self.sim.build_barnes_hut_tree()
            
        if self.collision_detection.get() == "Collision detection: Barnes-Hut":
            self.sim.barnes_hut_collision_merge()
        else:
            self.sim.direct_sum_collision_merge()
        
        if self.stepping_mode.get() == "Stepping method: Direct Sum":
            self.sim.direct_sum_step()
        else:
            self.sim.barnes_hut_step()
            
        self.refresh_selected_body_info()
        
        
    def init_tkinter(self):
        # Configuring basic settings
        self.root.configure(background="white", highlightcolor="white", highlightthickness=1)
        self.root.geometry(f"250x{self.height}+0+0")
        self.root.resizable(False, False)
        self.root.overrideredirect(True)
        self.root.attributes('-topmost', True)
        
        basic_buttons = Frame(self.root, background="white")
        Button(basic_buttons, text="Quit", command=self.kill).grid(row=0, padx=5)
        Button(basic_buttons, text="Pause", command=self.un_pause_or_pause).grid(row=0, column=1, padx=5)
        Button(basic_buttons, text="Step", command=self.manual_step).grid(row=0, column=2, padx=10)
        Button(basic_buttons, text="Add body", command=lambda:self.invert_adding_body()).grid(row=0, column=3)
        basic_buttons.grid(sticky="W")
        
        modes = Frame(self.root, background="white")
        Button(modes, textvariable=self.stepping_mode, command=lambda:self.change_stepping_method()).grid(row=0, column=0)
        Button(modes, textvariable=self.collision_detection, command=lambda:self.change_collision_detection_method()).grid(row=1, column=0)
        Label(modes, background="white").grid()
        modes.grid(sticky="W")
        
        slider_labels = Frame(self.root, background="white")
        Label(slider_labels, text="     FPS             Timestep       Arrow length", background="white").grid(row=0, column=0)
        slider_labels.grid(sticky="W")
        
        sliders = Frame(self.root, background="white")
        fps_slider = Scale(sliders, command=self.set_fps, orient="vertical", from_=1, to=1000, resolution=1)
        fps_slider.set(self.fps)
        fps_slider.grid(row=0, column=0, padx=(0, 15))
        
        time_step_slider = Scale(sliders, command=self.set_timestep, orient="vertical", from_=0.01, to=100, resolution=0.01)
        time_step_slider.set(self.sim.t)
        time_step_slider.grid(row=0, column=1, padx=(0, 15))
        
        arrow_scale_slider = Scale(sliders, command=self.set_arrow_scale, orient="vertical", from_=0.01, to=100, resolution=0.01)
        arrow_scale_slider.set(self.arrow_scale)
        arrow_scale_slider.grid(row=0, column=2, padx=5)
        
        Label(sliders, text="theta").grid(row=1, column=1)
        
        arrow_scale_slider = Scale(sliders, command=self.set_theta, orient="horizontal", from_=0.01, to=2, resolution=0.01)
        arrow_scale_slider.set(self.sim.θ)
        arrow_scale_slider.grid(row=2, column=1, sticky="W")
        
        Label(sliders, text="epsilon").grid(row=3, column=1)
        
        arrow_scale_slider = Scale(sliders, command=self.set_epsilon, orient="horizontal", from_=0.01, to=10, resolution=0.01)
        arrow_scale_slider.set(self.sim.θ)
        arrow_scale_slider.grid(row=4, column=1, sticky="W")
        
        Label(sliders, background="white").grid(row=1)
        sliders.grid(sticky="W")
        
        body_info_frame = Frame(self.root, background="white")
        Label(body_info_frame, text="Selected body information").grid(padx = 25)
        
        for i,v in enumerate((("name", "Name:    "), ("m", "Mass (kg):     "), ("r", "Radius (m):     "), 
                              ("x", "x-coordinate:    "), ("y", "y-coordinate:    "), ("is_immobile", "is immobile?:    "), 
                              ("v", "Velocity (m s⁻¹):    "), ("vx", "x velocity (m s⁻¹):    "), 
                              ("vy", "y velocity (m s⁻¹):    "), ("a", "acceleration (m s⁻²):    "))):
            field = Label(body_info_frame, text=v[1])
            field.grid(row=i+1, column=0, sticky="W", padx=(10, 0))
            field.update()
            temp = Entry(body_info_frame, textvariable=self.selected_body_info[v[0]], relief="solid", width=(250-field.winfo_width())//7)
            temp.bind("<FocusIn>", lambda event:self.pause())
            temp.bind("<Return>", lambda event, field=v[0]:self.set_selected_body_field(field))
            temp.grid(row=i+1, column=0, padx=(field.winfo_width(), 0), sticky="W")
            self.body_info_entries[v[0]] = temp
        
        Button(body_info_frame, text="Delete selected body", command=lambda:self.delete_selected_body()).grid(row=i+2, sticky="W")
        Button(body_info_frame, text="Center on selected body", command=lambda:self.center_on_selected_body()).grid(row=i+3, sticky="W")
        Button(body_info_frame, text="De-center on selected body", command=lambda:self.decenter_on_selected_body()).grid(row=i+4, sticky="W")

        Label(body_info_frame, background="white").grid(row=i+5)
        body_info_frame.grid(sticky="W")
                     
        orbit_generator_frame = Frame(self.root, background="white")
        orbit_generator_frame.grid()
        Label(orbit_generator_frame, text="Stable orbit generator").grid(padx=50)
        
        for i,v in enumerate((("orbiting_body", "Orbiting body:   "), ("body_to_orbit_around", "Body to orbit around:   "))):
            field = Label(orbit_generator_frame, text=v[1])
            field.grid(column=0, sticky="W", padx=(10,0))
            field.update()
            temp = OptionMenu(orbit_generator_frame, self.orbit_generator_inputs[v[0]], [])
            temp.grid(row=i+1, column=0, padx=(field.winfo_width(), 0), sticky="W")
            self.orbit_generator_optionmenus[v[0]] = temp
            
        field = Label(orbit_generator_frame, text="Semi-major axis length (m):    ")
        field.grid(column=0, sticky="W", padx=10)
        field.update()
        temp = Entry(orbit_generator_frame, textvariable=self.orbit_generator_inputs["semimajor_axis_length"], relief="solid", width=10)
        temp_var = self.orbit_generator_inputs["semimajor_axis_length"]
        temp.bind("<FocusIn>", lambda event:self.pause())
        temp.bind("<Return>", lambda event: temp_var.set(temp.get() if temp.get().replace(".", "").isnumeric() else "Numeric input only"))
        temp.grid(row=3, column=0, sticky="W", padx=(field.winfo_width(), 0))
        
        Button(orbit_generator_frame, text="Generate stable orbit", command=lambda:self.gen_orbit_from_inputs()).grid()
                
    
    def set_theta(self, event):
        self.sim.θ = float(event)
        
        
    def set_epsilon(self, event):
        self.sim.epsilon = float(event)
    
    
    def change_stepping_method(self):
        if self.stepping_mode.get() == "Stepping method: Direct Sum":
            self.stepping_mode.set("Stepping method: Barnes-Hut")
        else:
            self.stepping_mode.set("Stepping method: Direct Sum")  
            
    
    def change_collision_detection_method(self):
        if self.collision_detection.get() == "Collision detection: Barnes-Hut":
            self.collision_detection.set("Collision detection: Direct Sum")
        else:
            self.collision_detection.set("Collision detection: Barnes-Hut")  
    
    
    def manual_step(self):
        self.step()
        self.refresh_selected_body_info()    
    
        
    def decenter_on_selected_body(self):
        if self.centered_body:
            b = self.centered_body
            self.centered_body = None
            self.x_offset = -b.x*self.body_scaling
            self.y_offset = b.y*self.body_scaling
            
        
    def center_on_selected_body(self):
        if self.selected_body:
            self.centered_body = self.selected_body
            self.x_offset, self.y_offset = 0, 0
        
        
    def delete_selected_body(self):
        if self.selected_body is None: return
        body_names = [b.name for b in self.sim.bodies]
        bi = body_names.index(self.selected_body.name)
        del self.sim.bodies[bi]
        self.sim.num_bodies -= 1
        self.selected_body = None
        self.refresh_selected_body_info()
        self.draw_bodies()
        
        
    def invert_adding_body(self):
        self.adding_body = not self.adding_body
        
        
    def gen_orbit_from_inputs(self):
        valid = True
        for i in self.orbit_generator_inputs:
            if i == "-" or i == "Numeric input only":
                valid = False
                
        if valid:
            body_names = [b.name for b in self.sim.bodies]
            orbiting_body_name = self.orbit_generator_inputs["orbiting_body"].get()
            body_to_orbit_around_name = self.orbit_generator_inputs["body_to_orbit_around"].get()
            semi_major_axis_length = float(self.orbit_generator_inputs["semimajor_axis_length"].get())
            b1 = self.sim.bodies[body_names.index(orbiting_body_name)]
            b2 = self.sim.bodies[body_names.index(body_to_orbit_around_name)]
            try:
                self.sim.generate_stable_orbit(b1, b2, semi_major_axis_length)
                for i in self.orbit_generator_inputs.values():
                    i.set("-")
            except ValueError:
                self.orbit_generator_inputs["semimajor_axis_length"].set("Too small!")

        
    def set_selected_body_field(self, field):
        entry_widget = self.body_info_entries[field]
        inputted_value = entry_widget.get()
        b = self.selected_body
        if field not in ("name", "is_immobile"): # The input must be numeric
            try:
                new_value = float(inputted_value)
                if field == "v" and (b.vx**2+b.vy**2) != 0:
                    k = math.sqrt(new_value**2/(b.vx**2+b.vy**2)) # Scale factor for the x and y velocities
                    exec(f"b.vx *= {k}")
                    exec(f"b.vy *= {k}")
                elif field == "a":
                    pass
                else:
                    exec(f"b.{field} = {new_value}")
            except ValueError: # Incorrect input
                pass
        elif field == "is_immobile":
            if inputted_value.lower() in ("true", "false"):
                exec(f"b.{field} = {inputted_value.lower().capitalize()}")
            else:
                pass
        else:
            if inputted_value not in [b.name for b in self.sim.bodies]:
                exec(f"b.name = '{inputted_value}'")
        self.refresh_selected_body_info()

       
    def set_fps(self, event):
        self.fps = int(event)
    
    
    def set_timestep(self, event):
        self.sim.t = float(event)
        
        
    def set_arrow_scale(self, event):
        self.arrow_scale = float(event)**2
        
    
    def kill(self):
        self.root.destroy()
        pygame.quit()
    
    
    def pause(self):
        self.paused = True
    
    
    def un_pause_or_pause(self):
        self.paused = not self.paused
    

    def gui_mainloop(self):
        while True:
            try:
                self.window.fill("black") # Clear the window
                self.check_events()
                self.draw_grid()
                self.refresh_names_dropdown()
                self.draw_bodies()
                if not self.paused:
                    self.step()
                self.root.update()
                pygame.display.update()
                self.clock.tick(self.fps)
            except pygame.error:
                break
            
            
    def draw_grid(self):
        grid_x_offset = self.width//2-(self.width//(2*self.square_length))*self.square_length
        grid_y_offset = self.height//2-(self.height//(2*self.square_length))*self.square_length
        
        v = (-self.x_offset-self.width/2)/self.square_length
        v2 = abs(math.trunc(v))-abs(v)
        if v > 0: v2 *= -1
        grid_x_offset = -int(v2*self.square_length)
        
        v = (-self.y_offset-self.height/2)/self.square_length
        v2 = abs(math.trunc(v))-abs(v)
        if v > 0: v2 *= -1
        grid_y_offset = -int(v2*self.square_length)
        
        # Draw small grid squares
        for w in range(grid_x_offset, self.width, self.square_length):
            pygame.draw.line(self.window, "#191919", (w, 0), (w, self.height))
        for h in range(grid_y_offset, self.height, self.square_length):
            pygame.draw.line(self.window, "#191919", (0, h), (self.width, h))
            
        # Draw large grid squares, containing 5x5 small grid squares
        self.large_square_length = self.square_length*5
        grid_x_offset = self.width//2-(self.width//(2*self.large_square_length))*self.large_square_length
        grid_y_offset = self.height//2-(self.height//(2*self.large_square_length))*self.large_square_length
        
        v = (-self.x_offset-self.width/2)/self.large_square_length
        v2 = abs(math.trunc(v))-abs(v)
        if v > 0: v2 *= -1
        grid_x_offset = -int(v2*self.large_square_length)
        
        v = (-self.y_offset-self.height/2)/self.large_square_length
        v2 = abs(math.trunc(v))-abs(v)
        if v > 0: v2 *= -1
        grid_y_offset = -int(v2*self.large_square_length)
        
        for w in range(grid_x_offset, self.width, self.large_square_length):
            pygame.draw.line(self.window, "#444444", (w, 0), (w, self.height))
        for h in range(grid_y_offset, self.height, self.large_square_length):
            pygame.draw.line(self.window, "#444444", (0, h), (self.width, h))
            
        # Draw center lighter grid lines
        pygame.draw.line(self.window, "#BBBBBB", (self.width//2+self.x_offset, 0), 
                        (self.width//2+self.x_offset, self.height))
        pygame.draw.line(self.window, "#BBBBBB", (0, self.height//2+self.y_offset), 
                        (self.width, self.height//2+self.y_offset))
            

    def refresh_selected_body_info(self):
        cx, cy = 0, 0
        if self.centered_body:
            cx = self.centered_body.x
            cy = self.centered_body.y
        
        b = self.selected_body
        if b is None:
            for i in self.selected_body_info.keys():
                self.selected_body_info[i].set("-") 
        else:
            b.v = math.sqrt(b.vx**2+b.vy**2)
            assignments = {"name": b.name, "m": f"{b.m:.3e}", "r": f"{b.r:.3e}", "x": round(b.x-cx), "y": round(b.y-cy),
                           "is_immobile": b.is_immobile, "v": f"{b.v:.3e}", "vx": f"{b.vx:.3e}", "vy": f"{b.vy:.3e}", 
                           "a": f"{b.a:.3e}", "ax": f"{b.ax:.3e}", "ay": f"{b.ay:.3e}", "a": f"{b.a:.3e}"}
            
            for i in self.selected_body_info.keys():
                self.selected_body_info[i].set(str(assignments[i]))
                
            
    def refresh_names_dropdown(self):
        menu1 = self.orbit_generator_optionmenus["orbiting_body"]["menu"]
        menu1_var = self.orbit_generator_inputs["orbiting_body"]
        menu1.delete(0, menu1.index(END))
        [menu1.add_command(label=b.name, command=lambda name=b.name: menu1_var.set(name)) for b in self.sim.bodies]
        
        menu2 = self.orbit_generator_optionmenus["body_to_orbit_around"]["menu"]
        menu2_var = self.orbit_generator_inputs["body_to_orbit_around"]
        menu2.delete(0, menu2.index(END))
        for b in self.sim.bodies:
            if b.name != self.orbit_generator_inputs["orbiting_body"].get():
                menu2.add_command(label=b.name, command=lambda name=b.name: menu2_var.set(name))
        
        if menu2_var.get() == menu1_var.get():
            menu2_var.set("-")
                        
            
    def check_events(self):
        for event in pygame.event.get():
            if event.type == pygame.MOUSEWHEEL:
                self.body_scaling *= 1+event.y*0.1 # Zoom is increased or decreased by 10% on zoom
                self.x_offset *= 1+event.y*0.1
                self.y_offset *= 1+event.y*0.1                
                
                self.square_length = int(self.square_length*(1+event.y*0.1))
                if self.square_length < 10: self.square_length = 30
                elif self.square_length > 30: self.square_length = 10
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button not in (4, 5):
                centering_offset_x = 0
                centering_offset_y = 0
                if self.centered_body:
                    centering_offset_x = self.centered_body.x*self.body_scaling
                    centering_offset_y = self.centered_body.y*self.body_scaling
                    
                screen_x, screen_y = event.pos
                # Convert screen pixel coordinates to simulation coordinates
                sim_x = (screen_x-self.width//2-self.x_offset+centering_offset_x)/self.body_scaling
                sim_y = (-screen_y+self.height//2+self.y_offset+centering_offset_y)/self.body_scaling

                if self.adding_body and screen_x > 275:
                    self.pause()
                    self.sim.add_body(10**10, 5, sim_x, sim_y, f"Body {self.sim.num_bodies+1}", False)
                    self.selected_body = self.sim.bodies[-1]
                else:  
                    self.mouse_held = True
                    # Check all bodies to see if one was clicked on
                    for b in self.sim.bodies:
                        # Check if the distance between the mouse click and the center of the body is <= radius
                        if math.dist((b.x, b.y), (sim_x, sim_y)) <= b.r:
                            if self.selected_body == b: self.selected_body = None
                            else: self.selected_body = b
                            break
                
                self.refresh_selected_body_info()
                    
            elif event.type == pygame.MOUSEBUTTONUP:
                self.mouse_held = False
            elif event.type == pygame.MOUSEMOTION and self.mouse_held:
                self.x_offset += event.rel[0]
                self.y_offset += event.rel[1]
                
                
    def draw_bodies(self):
        if self.adding_body:
            mouse_pos = pygame.mouse.get_pos()
            pygame.draw.circle(self.window, "blue", mouse_pos, 5)
            
        centering_offset_x = 0
        centering_offset_y = 0
        if self.centered_body:
            centering_offset_x = self.centered_body.x*self.body_scaling
            centering_offset_y = self.centered_body.y*self.body_scaling
        
        for b in self.sim.bodies:
            x = b.x*self.body_scaling+self.width//2+self.x_offset-centering_offset_x
            y = -b.y*self.body_scaling+self.height//2+self.y_offset+centering_offset_y
            color = "yellow" if b == self.selected_body else "blue"
            pygame.draw.circle(self.window, color, (x, y), math.ceil(b.r*self.body_scaling))
            if b == self.selected_body and not b.is_immobile: # Draw the vector arrow showing its direction of motion
                
                x1, y1 = x+b.vx*self.arrow_scale*self.body_scaling, y-b.vy*self.arrow_scale*self.body_scaling
                pygame.draw.line(self.window, "red", (x, y), (x1, y1), width=math.ceil(b.r*self.body_scaling*0.5+5))
                mag = max(math.sqrt(b.vx**2+b.vy**2), 0.000001)
                length = max(b.r*self.body_scaling, 10)
                k = math.sqrt(3)*length/mag
                pygame.draw.polygon(self.window, "red", ((x1+length/mag*b.vy, y1-length/mag*-b.vx),
                                                         (x1+length/mag*-b.vy, y1-length/mag*b.vx),
                                                         (x1+b.vx*k, y1-b.vy*k)))
                
                x2, y2 = x+b.ax*self.arrow_scale*self.body_scaling, y-b.ay*self.arrow_scale*self.body_scaling
                pygame.draw.line(self.window, "purple", (x, y), (x2, y2), width=math.ceil(b.r*self.body_scaling*0.5+5))
                mag = max(math.sqrt(b.ax**2+b.ay**2), 0.000001)
                length = max(b.r*self.body_scaling, 10)
                k = math.sqrt(3)*length/mag
                pygame.draw.polygon(self.window, "purple", ((x2+length/mag*b.ay, y2-length/mag*-b.ax),
                                                         (x2+length/mag*-b.ay, y2-length/mag*b.ax),
                                                         (x2+b.ax*k, y2-b.ay*k)))

            # Draw the ellipse for an orbit, if it is being made by the user
            body_names = [b.name for b in self.sim.bodies]
            orbiting_body_name = self.orbit_generator_inputs["orbiting_body"].get()
            body_to_orbit_around_name = self.orbit_generator_inputs["body_to_orbit_around"].get()
            semi_major_axis_length = self.orbit_generator_inputs["semimajor_axis_length"].get()
            if orbiting_body_name == b.name and body_to_orbit_around_name in body_names and semi_major_axis_length.replace(".", "").isnumeric():
                b1 = self.sim.bodies[body_names.index(orbiting_body_name)]
                b2 = self.sim.bodies[body_names.index(body_to_orbit_around_name)]
                semi_major_axis_length = a = float(semi_major_axis_length)
                
                # Calculate the semi-minor axis length
                θ = math.atan2(b2.y-b1.y, b2.x-b1.x) # Angle from orbiting body to the body it orbits around
                apoapsis = ra = math.dist((b1.x, b1.y), (b2.x, b2.y))
                periapsis = rp = math.dist((b1.x+math.cos(θ)*2*a, b1.y+math.sin(θ)*2*a), (b2.x, b2.y))
                orbit_eccentricity = (ra-rp)/(ra+rp)             
                semi_minor_axis_length = b = a*math.sqrt(1-orbit_eccentricity**2)
                
                # Draw and rotate the ellipse
                ecx, ecy = ellipse_center = (self.width//2+self.body_scaling*(b1.x+a*math.cos(θ)), 
                                             self.height//2-self.body_scaling*(b1.y+a*math.sin(θ)))
                ellipse_surface = pygame.Surface((2*a*self.body_scaling, 2*b*self.body_scaling), pygame.SRCALPHA)
                pygame.draw.ellipse(ellipse_surface, "green", (0, 0, 2*a*self.body_scaling, 2*b*self.body_scaling), 5)
                ellipse_surface = pygame.transform.rotate(ellipse_surface, math.degrees(θ))
                new_rect = ellipse_surface.get_rect()
                w, h = new_rect.size
                self.window.blit(ellipse_surface, (ecx-w/2+self.x_offset, ecy-h/2+self.y_offset))
                
                

In [58]:
#region
# To-do before analysis deadline:
    # Mention smoothening and barnes-hut in analysis, explain fully
    # Hide word in analysis
    # Do an example calculation with the ALL the maths
    # Add more diagrams
    # Show how it's currently taught
    # Ask Mr. Van Setten for a photo
    # Make little quiz to test user knowledge
    # Develop interview quiz
    # Model the algorithms
    # INTERVIEW about lagrange points and equipotential lines
    
# To-do before implementation deadline:
    # Add a user manual
    # Check comments, rework code for readability
    # Use circular queues

# OPTIONAL EXTRAS:
    # Keep temporary logs of velocity/other parameters and then graph them on-screen
    # Illustrate gravitational field around the body
    # More features (ask teachers)
    # Slight optimizations
#endregion

In [59]:
# random.seed(5)
# ns = list(range(10, 1000, 10))
# j = 0
# times_barnes = []
# times_barnes2 = []
# times_direct = []
# for n in ns:
#     if j%50== 0: print(j)
#     sim = GravitySimulation(1)
#     x = random.sample(range(1, n*2), n)
#     y = random.sample(range(1, n*2), n)

#     for i in range(n):
#         sim.add_body(1, 10, x[i], y[i], "Body", False)
        
#     j += 1
                
#     times_barnes.append(timeit.timeit(lambda:sim.barnes_hut_step(), number=1))
#     times_barnes2.append(timeit.timeit(lambda:sim.step2(), number=1))
#     times_direct.append(timeit.timeit(lambda:sim.direct_sum_step(), number=1))

In [60]:
# plt.scatter(x=ns, y=times_barnes)
# plt.scatter(x=ns, y=times_barnes2)
# plt.scatter(x=ns, y=times_direct)

In [61]:
# random.seed(5)
# ns = list(range(100, 10000, 10))
# times = []
# for n in ns:
#     sim = GravitySimulation(1)
#     x = random.sample(range(1, n*2), n)
#     y = random.sample(range(1, n*2), n)

#     for i in range(n):
#         sim.add_body(1, 10, x[i], y[i], "Body", False)
            
#     tree = BarnesHutTree()
#     times.append(timeit.timeit(lambda:tree.build(sim.bodies), number=1))

In [62]:
# plt.scatter(x=ns, y=times)

In [63]:
# random.seed(5)
# ns = list(range(100, 3000, 10))
# times_brute = []
# times_barnes = []
# for n in ns:
#     sim = GravitySimulation(1)
#     x = random.sample(range(1, n*2), n)
#     y = random.sample(range(1, n*2), n)

#     for i in range(n):
#         sim.add_body(1, 10, x[i], y[i], "Body", False)
        
#     sim.build_barnes_hut_tree()
            
#     times_brute.append(timeit.timeit(lambda:sim.direct_sum_collision_merge(), number=1))
#     times_barnes.append(timeit.timeit(lambda:sim.barnes_hut_collision_merge(), number=1))


In [64]:
# plt.scatter(ns[:-8], times_brute, c="red")
# plt.scatter(ns[:-8], times_barnes, c="yellow")

In [65]:
GUI()